## **WEEK 2**

- EDA
- Validation
- Data Leakage

## Exploratory Data Analysis(EDA)

### 概要

1. EDA とは何か？なぜするのか？
2. 探索するもの
3. 探索と可視化のツール
4. データセットクリーニング
5. Kaggle コンペにおける EDA

### EDAとは何か？なぜするのか？

- EDAとは
    - データをよく観察し、データをより良く理解すること
- EDAをする理由は
    - 強い特徴量を生み出し、正確なモデルを構築するため
    - データに関する直感を身につけるため
    - 新しい効果的な特徴量生成のための仮説を生み出すため
    - より良いスコアのためのインサイトを見つけるため
    
- 複雑なモデルは上位解法でよく見られ、重要ではあるが、一方で彼らは注意深くEDAを行っている。
    - モデリングに時間を割くよりも EDA は重要なプロセスである
    
- コンペはまず EDA から始めるのが良い。
    
- EDA において可視化が主要な手段の一つである。
    - 可視化によってパターンを一目で分かることができる
    
- [あるコンペ](https://www.kaggle.com/c/dunnhumbychallenge)では、モデリングなしにデータの観察のみで優勝した例がある
    - プロモーションを使用するかどうかを予測するコンペ
    - プロモーションが送られた累計と使われた累計の列があったので、ユーザーごとに累計をソートして差分を取ると分類機を使用せずとも80％程度の accuracy を出すことができる。
        - 当然にこれはデータリークと言えるが、コンペにおいては通常をこれを利用しても良いことになっている

### データへの直感を身につける

- ドメイン知識の取得
    - 問題をより深く理解するのに役立つ
- データが直感的かどうかを確認する
    - ドメイン知識と一致するかどうかを確認する
- どのようにデータが生成されたかを理解する
    - 適切なバリデーションセットを構築するために必須であるため

#### ドメイン知識の取得

- コンペには様々な領域のものがあり、しばしばドメイン知識を持っていないコンペに遭遇することがある
    - ドメインに対して深く知ることはないが、通常はどんな目的か・どんなデータを持っているか・どのように人々がベースラインを構築しようとしているかを知ることは重要である。
    - Google / Wikipedia を使ってデータの種類を理解することがまず第一歩となる
    
- データのドメインの理解・列名の理解
- データの理解後は、データが直感と一致しているかを確認する（ImpはClickより必ず大きいかとか、年齢が100歳以上なのは珍しいとか）
    - 年齢が336とかなら、タイポとして33歳か36歳に手動で訂正するのが通常良いが、念の為データ説明を読み直しておく
- 一方で、Imp が0なのに Click が3の場合などは通常のエラーであることがある
    - スクリプトから送られたデータなどには付きもののエラーでランダムではないことが多いので、これは特徴量として用いるのが良い
        - 'is_incorecct' などの列を作るなどして

- データがどのように生成されたかを知ることも重要
    - データベースからサンプリングされた場合、少数クラスがオーバーサンプリングされて抽出されている可能性がある
- データがどのように生成されたかが分かれば、適切なバリデーションセットを作ることができる。
- 学習データとテストデータが異なるアルゴリズムで生成されている場合、通常以下のような特徴が見られることが多い
    - CVとLBの連動しない
    - 各特徴量の分布が異なる
    - 各データセットの期間（日付等）の範囲が異なる

#### Reading Material for Building Intution about the Data

- https://hdinwtxpwdelvatxpgqiri.coursera-apps.org/notebooks/readonly/reading_materials/EDA_video2.ipynb

### 匿名データを探索する

- 匿名データとは？
    - なぜホストはデータを匿名化するのか
- 匿名データに対して何ができるか？
    - 特徴量の真の意味を推測し、decode してみる
    - どのような前処理が必要になるかを知るために、特徴量のデータ型を推測してみる
    
#### 匿名データとは

- 機械学習モデルを構築したいけど、データそのものを公表したくないときにデータ匿名化を行うことがある
    - テキストデータがハッシュ関数を通して暗号化する
    - テーブルデータでの列名をダミー化する
- テーブルデータの列匿名化に対して
    - 個別の特徴量に対して
        - 列の真の意味を推測する
        - 列のデータ型を推測する
    - 特徴量の関係に対して
        - 特徴量ペアの関連を見つける
        - 特徴量のグループを見つける
        
- [個別の特徴量に対してのアプローチの例](https://hdinwtxpwdelvatxpgqiri.coursera-apps.org/notebooks/readonly/reading_materials/EDA_video3_screencast.ipynb)
    - Baseline Model(RF)を構築
    - RF の Feature Importance を見て、x8 が重要度が高そうなので探索する
        - 平均と分散の確認（ここでは正規分布であることが分かった）
        - そのため、元の分布に戻したくなる（scale back）
    - value_counts() で繰り返し現れる値を確認
        - 出現する数値に偏りがありそうなことがわかった
        - 例えば、ユニークな値のみ取り出し、それをソートして、差分を取る
            - 多くの差分が同じ値になった(0.043)
            - 差分の元の正確な値が分からなくても、例えばそれを1.0と想定してみる(差分を0.043で割る)
    - 元の値をそれで割ってみる
        - 小数点以下で同じような数値(1.02468)が並ぶので、これを引いてやる
        - (train.x8 - 1.02468) / 0.043 として、これを丸め込むと良い感じの整数が出てくる
    - スケールバックしたら、再びその値の value_counts() を見てみる
        - 最小値に -1968 があったので、西暦っぽいという仮説が成り立つ（ユーザーがテキストボックスに適当に数値を入れた？）
        - 1968 を足すと誕生年っぽい感じになる
        - このスケールバックした特徴量を用いて、年齢グループなどの新しい特徴量を生成できうる
        - （実際にコンペが終了した後、ホストがこの列は誕生年だということを発表した）
        
#### データ型を推定する

- 目視で見ていく
- あるいは以下の関数を用いる
    - `df.dtypes`
    - `df.info()`
    - `x.value_counts()`
    - `x.isnull()`

### Visualizations

データから興味深いものを見つけるのに万能な方法はない。そのため、EDA はある種のアートであり、可視化はそれを実現するためにツールである。
可視化は以下の目的のために行う。

- 個別の特徴量の探索
    - ヒストグラム
    - プロット
    - 統計量
- 特徴量の関係性の探索
    - 散布図
    - 相関図
    - プロット（インデックスと特徴量の統計量の比較）
    

#### 個別の特徴量の探索

**Histogram**

`plt.hist(x)`

- 特徴量を bins で区切って、各々の bins にいくつの点が含まれるかを示すグラフ
    - bins の区切り方にとって誤解を生むことがあるので、bins の区切り方はいくつか試してみると良い。
        - ヒストグラムはデータを集計するので、値の詳細を見ることができないので注意が必要。
        - ある一つのプロットのみで結論をだすことは誤った結論を導くことになりうる。
        - 仮説を立てたら、複数のプロットを使ってそれを実証することが重要である。
    - ヒストグラムは大きなピークを持つことがある
        - これはホストが欠損値を平均や適当な値で埋めた形跡である可能性が高く、値は元々欠損値だったと推測される。
        - この場合、以下のような方法でこの情報を用いることができる。
            - 値を再び NaN に置き換えて、欠損値を上手に処理する XGBoost などを使用する
            - 平均値ではなく、-999などの値で埋める
            - 欠損値かどうかを示す新しい特徴量を追加する（線形モデルで特に有効）
            
**Plot (index vs. value)**

`plt.plot(x, '.')`

- X軸に行のインデックス、Y軸に列の値を取るプロット
    - 線ではなく、点でプロットした方が、独立した点で見れるので便利である。
    - 水平方向に線があることが見られるので、特徴量の中で値が多く繰り返し出現しているが分かる。
    - 垂直方向にはパターンが見られないので、データが適切にシャッフルされていることが分かる。

`plt.scatter(range(len(x)), x, c=y)`

- ラベルごとに色付けすることもできる

**特徴量の統計量**

```
df.describe()  
x.mean()  
x.var()  
```

**ユニークな値のカウント**

```
x.value_counts()  
x.isnull()  
```


#### 特徴量の関係性の探索

複数の特徴量のペアの関係を見ることも有効である（特徴量のグループ化を目指す）

**散布図**

`plt.scatterplot(x1, x2)`

- 分類問題の場合、正解ラベルのよって点を色付けするのが便利である。
- 回帰問題の場合、ヒートマップ色（グラデーション）で色付けするか、目的変数に比例した点のサイズで表現するのが便利である。

- 学習データとテストデータの2特徴量で散布図を作成した時に分布が大きくハズレている場合、コードがバグってるとか、オーバーフィットした特徴量であるとかなどで、よく考える必要がある。

- 散布図で X1 と X2 にある関係を見出したら、それを新たな特徴量として用いるのが良い。
    - ツリーベースモデルの場合ならば、X1 と X2 の差分や比率を特徴量とすることが考えられる。
    
- 特徴量の数が少なかったら、 `pd.scatter_matrix(df)` を使うといっぺんに全ての特徴量間の散布図を表示できるので便利。

**相関図**

`df.corr()`, `plt.matshow( ... )`

- 特徴量間の「距離」を計測できる
- 特徴量間の独自の距離を計算したいときは、plt.matshow() で独自関数を渡して「距離」を可視化できる
- K-meansクラスタリングのようなクラスタリング手法も用いることができる。これにより、特徴量をグループ化することができる。

**特徴量の統計量を比較する**

`df.mean().plot(style=',')`

- 各特徴量の統計量（平均値）を計算して、X軸に特徴量、Y軸に特徴量の統計量を取ると、統計量をグループ化する手がかりになる。

### データクリーニング・その他チェック事項

- データクリーニング
    - 定常の特徴量
    - 重複の特徴量
- その他チェック事項
    - 重複行
    - データセットがシャッフルされているかを確認
    
#### データクリーニング

- すべて一定の特徴量を見つける
    - `traintest.nunique(axis=1) == 1` で True のものは列の値がすべて同一なので不要な列である
    - `train.nunique(axis=1) == 1` で True の場合も、一般的にはその列は取り除くべきである。
        - 特に線形モデルの場合は、書く特徴量に重みを与えるためモデルが不安定になる可能性が高い
        - ただし、その列のテストデータのみに現れる値が重要かどうかを判断して慎重に扱う必要がある。
        
- 重複した特徴量を見つける
    - 学習スピードを下げるだけので不要である
        - `traintest.T.drop_duplicates()` で取り除くことができる
    - また、カテゴリカル変数で違うレベルで同じ値が振られている場合は問題になりうるので、Identical にする必要がある
        - `for f in categorical_feats: traintest[f] = traintest[f].facotrize()`
        - 'traintest.T.drop_duplicates()`
            
### その他チェック事項

- 重複行
    - 同じ特徴量を持った行が同じ正解ラベルを持っているかどうかを確認する
        - 同じ特徴量を持った行が異なる正解ラベルを持っている場合、そのコンペはガチャ的なものになるか、あるいはその重複行が単なるミスかもしれない。
        - また、学習データとテストデータの間に共通の行があるかを確認すべきである。時々、これによってデータ生成のプロセスが示唆されることがある。その理由を考えることも重要である。
        
- データセットが適切にシャッフルされているかを確認する
    - データセットが適切にシャッフルされていない場合、データリークを見つけることができる可能性がある。
    - 特徴量あるいはTarget ValueをY軸に、行のインデックスをX軸に取ることで見つけることができうる。
        - Rolling-mean を時に使用することもある。
        - Rolling-mean は全体の平均の付近を行き来することを期待するが、そうならないときには何かしらの情報が隠れていることを見つけることができる。
        
- あらゆる側面から可視化を行うことで、Magic Features を見つけられるようになる。

### Additional Materials and Links

#### Visualization tools
- [Seaborn](https://seaborn.pydata.org/)
- [Plotly](https://plot.ly/python/)
- [Bokeh](https://github.com/bokeh/bokeh)
- [ggplot](http://ggplot.yhathq.com/)
- [Graph visualization with NetworkX](https://networkx.github.io/)

#### Others
- [Biclustering algorithms for sorting corrplots](https://scikit-learn.org/stable/auto_examples/bicluster/plot_spectral_biclustering.html)
